#### Jennifer Camacho, Octavia Larentis and Lauren Picado - 18 May, 2019
-------------------------------------------------------------------------------------------------------------------------------
# Robust Principal Component Analysis (RPCA) for detecting Moving Foreground in a Video Sequence
### Implemented with the Principal Component Pursuit (PCP) and Stable Principal Component Pursuit (SPCP)

#### Importing required Python libraries

In [1]:
# Processing and visualization
from glob import glob
from IPython import display
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os, sys
sys.path.insert(0, os.path.dirname(os.path.abspath('..')))
import scipy.misc
import shutil
import subprocess
import time
import tkinter

# Image Processing
from PIL import Image, ImageTk
from PIL import Image
import cv2

# Statistical Analysis
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

### RCPA Methods

#### PCP-RPCA

In [2]:
def rpca_pcp(M, lambda0="default", convergenceTolerance=10**(-3), convergenceLog="pcp"):
              
    # Define a maximum number of iterations
    maximumIterations=1000
    # Frame size
    height, width = M.shape
    # Set all empty values to 0
    tmp = np.isnan(M)
    M[tmp] = 0
    
    # Prepare a low-rank matrix to store background output
    L = np.zeros((height, width))  
    # Prepare a sparse matrix to store the moving object 
    S = np.zeros((height, width))
   
    dual_variable = np.zeros((height, width)) # Dualvar update
    
    # Set default penalty term lambda
    if lambda0 == "default":
        lambda0 = 1/np.sqrt(max(height, width)) * float(1)
        
    # Set defaults
    mu0 = 0.25/np.abs(M).mean()

    # Open log files for convergence criteria
    f = open(convergenceLog + "_ds.txt", "a")
    g = open(convergenceLog + "_dl.txt", "a")
        
    for iterations in range(0, maximumIterations):
        
        # Current Euclidean distance between the sparse and low-rank matrix
        euclid = np.linalg.norm(np.concatenate((S,L), axis=1), 'fro')              
        temp_X = dual_variable / mu0 + M
        temp_Y = temp_X - S;
        
        # Note the change to the current low-rank and sparse matrices
        change_L = L;   
        change_S = S
        temp_U, temp_sigma, temp_V = np.linalg.svd(temp_Y, full_matrices=False);
        
        # Calculate matrix rank
        matrixRank = (temp_sigma > 1/mu0).sum()
        sigma_iter = np.diag(temp_sigma[0:matrixRank] - 1/mu0)
        
        # Iterative update of the low-rank matrix
        L = np.dot(np.dot(temp_U[:,0:matrixRank], sigma_iter), temp_V[0:matrixRank,:])
        
        # Update change to low-rank matrix
        change_L = L - change_L
        
        temp_Y = temp_X - L
        S = soft_threshold(temp_Y, lambda0/mu0)
        change_S = S - change_S
        
        # Dualvar change
        temp_Z = M - S - L
        temp_Z[tmp] = 0
        dual_variable = dual_variable + mu0 * temp_Z;
        
        # Check whether the convergence condition has been met
        stoppingCriterion = np.linalg.norm(np.concatenate((change_S, change_L), axis=1), 'fro') / (euclid + 1)
        
        # Log the current state of low-rank and sparse matrices
        f.write(np.array2string(change_S))
        g.write(np.array2string(change_L))
         
        # Terminate if the convergence criterion has been reached
        if stoppingCriterion < convergenceTolerance: 
            break
    
    # Close log files
    f.close()
    g.close()
    
    return L, S

#### SPCP-RPCA

In [3]:
def rpca_spcp(M, lambda0="default", convergenceTolerance=10**(-3), convergenceLog="spcp"):
    
    # Define a maximum number of iterations
    maximumIterations=1000
    # Frame size
    height, width = M.shape
    # Default mu
    mu0 = np.sqrt(2 * max(height, width))
    # Set penalty lambda
    if lambda0=="default":
        lambda0 = 1.0 / np.sqrt(max(height, width))
        
    tmp_L = np.zeros((height, width)) 
    tmp_S = np.zeros((height, width))
    
    # Prepare a low-rank matrix to store background output
    L = np.zeros((height, width))
    # Prepare a sparse matrix to store the moving object 
    S = np.zeros((height, width))
    
    threshold_0 = 1
    threshold_1 = 1
    mu_iteration = mu0
    currentIteration = 1
    
    # Open log files for convergence criteria
    f = open(convergenceLog + "_ds.txt", "a")
    g = open(convergenceLog + "_dl.txt", "a")
    
    while 1:
        
        temp_Y_L = L + (threshold_0-1)/threshold_1*(L - tmp_L)
        temp_Y_S = S + (threshold_0-1)/threshold_1*(S - tmp_S)
        temp_G_L = temp_Y_L - 0.5*(temp_Y_L + temp_Y_S - M)
        
        # Use the Python numpy package for singular vector decomposition
        temp_U, temp_sigma, temp_V = np.linalg.svd(temp_G_L, full_matrices=False);
        
        # Calculate matrix rank
        matrixRank = (temp_sigma > mu_iteration/2).sum()
        sigma_iter = np.diag(temp_sigma[0:matrixRank] - mu_iteration/2)
        
        # Update change to low-rank matrix
        tmp_L = L
        L = np.dot(np.dot(temp_U[:,0:matrixRank], sigma_iter), temp_V[0:matrixRank,:])
        temp_G_S = temp_Y_S - 0.5*(temp_Y_L + temp_Y_S - M)
        
        # Update change to sparse matrix
        tmp_S = S
        S = soft_threshold(temp_G_S, lambda0*mu_iteration/2)
        threshold_1, threshold_0 = (np.sqrt(threshold_1**2+1) + 1)/2, threshold_1
        
        # Check against convergence tolerance
        change_L =2*(temp_Y_L - L) + (L + S - temp_Y_L - temp_Y_S)
        change_S =2*(temp_Y_S - S) + (L + S - temp_Y_L - temp_Y_S) 
        stoppingCriterion = np.sqrt(np.linalg.norm(change_L, ord='fro')**2 + np.linalg.norm(change_S, ord='fro')**2)
        
        # Log the current state of low-rank and sparse matrices
        f.write(np.array2string(change_S))
        g.write(np.array2string(change_L))
        
        if currentIteration >= maximumIterations or stoppingCriterion < convergenceTolerance:
            break
        else:
            currentIteration += 1
    
    # Close log files  
    f.close()
    g.close()
    
    return L, S

#### PCP-RPCA with decreasing penalty parameter

In [4]:
def rpca_pcp_vary(M, lambda0="default", convergenceTolerance=10**(-3), convergenceLog="pcp_vary"):
     
    # Define a maximum number of iterations
    maximumIterations=1000
    # Frame size
    height, width = M.shape
    tmp = np.isnan(M)
    M[tmp] = 0
    
    # Prepare a sparse matrix to store the moving object 
    S = np.zeros((height, width))
    # Prepare a low-rank matrix to store background output
    L = np.zeros((height, width))
    # Dualvar
    dual_variable = np.zeros((height, width)) 
 
    # Set default mu
    mu0 = 0.25/np.abs(M).mean()
    
    # Set penalty term lambda
    if lambda0 == "default":
        lambda0 = 1/np.sqrt(max(height, width)) * float(1)
    
    # Initiate the first mu term that will decrease with each iteration
    mu_iter = min(10*mu0, 0.99*np.linalg.norm(M, ord='fro'))   
    
    # Eta
    n = 0.8

    # Open log files for convergence criteria
    f = open(convergenceLog + "_ds.txt", "a")
    g = open(convergenceLog + "_dl.txt", "a")
       
    for iterations in range(maximumIterations):
        
        # Current Euclidean distance between the sparse and low-rank matrix
        euclid = np.linalg.norm(np.concatenate((S,L), axis=1), 'fro')   
        
        temp_X = dual_variable / mu_iter + M
        temp_Y = temp_X - S;
        
        # Temporarily assign current low-rank matrix
        change_L = L;       
        temp_U, temp_sigma, temp_V = np.linalg.svd(temp_Y, full_matrices=False);
        
        # Calculate matrix rank
        matrixRank = (temp_sigma > 1/mu_iter).sum()
        sigma_iter = np.diag(temp_sigma[0:matrixRank] - 1/mu_iter)
        
        # Update the low-rank matrix 
        L = np.dot(np.dot(temp_U[:,0:matrixRank], sigma_iter), temp_V[0:matrixRank,:])
        change_L = L - change_L
        temp_Y = temp_X - L
        change_S = S
        
        # Update the sparse matrix
        S = soft_threshold(temp_Y, lambda0/mu_iter)
        change_S = S - change_S

        # Dual variable update
        temp_Z = M - S - L
        temp_Z[tmp] = 0
        dual_variable = dual_variable + mu_iter * temp_Z;
        
        # Update mu each iteration
        mu_iter = max(n*mu_iter, mu0)
        
        # Log the current state of low-rank and sparse matrices
        f.write(np.array2string(change_S))
        g.write(np.array2string(change_L))
        
        # Check against convergence tolerance
        stoppingCriterion = np.linalg.norm(np.concatenate((change_S, change_L), axis=1), 'fro') / (euclid + 1)
        
        # Terminate if convergence has been reached
        if stoppingCriterion < convergenceTolerance: 
            break
    
    # Close log files
    f.close()
    g.close()
    
    return L, S

#### SPCP-RPCA with decreasing penalty parameter

In [45]:
def rpca_spcp_vary(M, lambda0="default", convergenceTolerance=10**(-3), convergenceLog="spcp_vary"):
    
    # Define a maximum number of iterations
    maximumIterations=1000
    # Frame size
    height, width = M.shape
    # Set default mu
    mu = np.sqrt(2*max(height, width))
    # Set default penalty term lambda
    if lambda0=="default":
        lambda0 = 1.0/np.sqrt(max(height, width))
    
    # Initiate the first mu term that will decrease with each iteration
    mu0 = min(10*mu, 0.99*np.linalg.norm(M, ord='fro'))   
    
    # Eta
    n = 0.8
 
    temp_L = np.zeros((height, width)) 
    temp_S = np.zeros((height, width))
    
    # Prepare a low-rank matrix to store background output
    L = np.zeros((height, width)) 
    # Prepare a sparse matrix to store the moving object
    S = np.zeros((height, width))
    
    threshold_0 = 1
    threshold_1 = 1
    mu_iteration = mu0
    currentIteration = 1
    
    # Open convergence log files
    f = open(convergenceLog + "_ds.txt", "a")
    g = open(convergenceLog + "_dl.txt", "a")
    
    while 1:
        
        # Y _k for kth iteration
        temp_Y_L = L + (threshold_0 - 1)/threshold_1*(L - temp_L)
        temp_Y_S = S + (threshold_0 - 1)/threshold_1*(S - temp_S)
    
        temp_G_L = temp_Y_L - 0.5*(temp_Y_L + temp_Y_S - M)
        
        # Use the Python numpy package for singular vector decomposition
        temp_U, temp_sigma, temp_V = np.linalg.svd(temp_G_L, full_matrices=False);
        
        # Calculate matrix rank
        matrixRank = (temp_sigma > mu_iteration/2).sum()
        sigma_iter = np.diag(temp_sigma[0:matrixRank] - mu_iteration/2)
        temp_L = L
        
        # Update the low rank matrix
        L = np.dot(np.dot(temp_U[:,0:matrixRank], sigma_iter), temp_V[0:matrixRank,:])
        temp_G_S = temp_Y_S - 0.5*(temp_Y_L + temp_Y_S - M)
        temp_S = S
        
        # Update the sparse matrix
        S = soft_threshold(temp_G_S, lambda0*mu_iteration/2)
        threshold_1, threshold_0 = (np.sqrt(threshold_1**2+1) + 1)/2, threshold_1
        
        # Update the mu term each iteration
        mu_iteration = max(n*mu_iteration, mu)
        
        # Current changes to the sparse and low-rank matrices
        change_L =2*(temp_Y_L - L) + (L + S - temp_Y_L - temp_Y_S)
        change_S =2*(temp_Y_S - S) + (L + S - temp_Y_L - temp_Y_S) 
        
        # Euclidean distance between updates to sparse and low-rank matrices
        euclid = np.sqrt(np.linalg.norm(change_L, ord='fro')**2 + np.linalg.norm(change_S, ord='fro')**2)
        
        # Log the current state of low-rank and sparse matrices
        f.write(np.array2string(change_S))
        g.write(np.array2string(change_L))
        
        # Check against convergence tolerance
        if currentIteration >= maximumIterations or euclid < convergenceTolerance:
            break
        else:
            currentIteration += 1
    
    # Close convergence log files
    f.close()
    g.close()
            
    return L, S

#### Soft Thresholding

In [6]:
def soft_threshold(inputs, mu0):
    
    # Check whether inputs is greater than mu operator
    outputs = np.maximum(inputs - mu0, 0)
    
    # Increment output by the difference
    outputs = outputs + np.minimum(inputs + mu0, 0)
    
    return outputs

### Pre-Processing Methods

In [7]:
def generateMatrix(frames):
    matrixVolume = []
    
    # Loop through frames and append to volume object
    for frame in frames:
        img = Image.open(frame).convert('L')
        dat = list(img.getdata())
        matrixVolume.append(dat)
    
    stackedFrames = np.array(matrixVolume)
    
    return stackedFrames

In [8]:
def writeImages(folder, filename, matrix, width, height):
    
    outputHeight, outputWidth = matrix.shape
    
    # Write individual frames for analysis
    for i in range(outputHeight):
        frame = matrix[i,:].reshape((width, height), order='F').transpose()   
        cv2.imwrite(folder + 'bin' + str(i+1).zfill(6) + '.png', frame)

In [9]:
def writeMask(folder, filename, matrix, width, height, out_threshold):
    
    outputHeight, outputWidth = matrix.shape
    
    # Create binary mask images
    for i in range(outputHeight):
        frame = matrix[i,:].reshape((width, height), order='F').transpose()
        tmp = frame
        
        # Splice at desired binary threshold level to create masks
        frame[tmp > out_threshold] = 255 
        frame[tmp < out_threshold] = 0 
        
        # Write frames to a folder for statistical analysis
        cv2.imwrite(folder + 'bin' + str(i+1).zfill(6) + '.png', frame)           

In [10]:
def convergencePlot(convergence_ds, convergence_dl):
    n = len(convergence_ds)
    iteration_ds = np.arange(1,(n + 1))
    m = len(convergence_dl)
    iteration_dl = np.arange(1, (m + 1))
    plt.figure(1)
    plt.plot(iteration_ds,convergence_ds,'r.')
    plt.figure(2)
    plt.plot(iteration_dl,convergence_dl,'b.')

### Pre-Processing Video Frames
-------------------------------------------------------------------------------------------------------------------------------

#### Highway Video

In [11]:
join = map(lambda s: os.path.join("..\\highway\\", s), os.listdir("..\\highway\\"))
highway = generateMatrix(join)

#### Pedestrian Video

In [12]:
join = map(lambda s: os.path.join("..\\pedestrian\\", s), os.listdir("..\\pedestrian\\"))
pedestrian = generateMatrix(join)

#### Office Video

In [13]:
join = map(lambda s: os.path.join("..\\office\\", s), os.listdir("..\\office\\"))
office = generateMatrix(join)

### Pre-Processing Ground Truth data

#### Highway Video

In [14]:
join = map(lambda s: os.path.join("..\\highway_groundtruth\\", s), os.listdir("..\\highway_groundtruth\\"))
highway_groundtruth = generateMatrix(join)

try:
    os.stat('..\\highway_G\\')
except:
    os.mkdir('..\\highway_G')

# Fix ground truth, remove multi-level segmentation 
writeMask("..\\highway_G\\", "highway", highway_groundtruth, 320, 240, 180)

join = map(lambda s: os.path.join("..\\highway_G\\", s), os.listdir("..\\highway_G\\"))
highway_g = generateMatrix(join)

#### Pedestrian Video

In [15]:
join = map(lambda s: os.path.join("..\\pedestrian_groundtruth\\", s), os.listdir("..\\pedestrian_groundtruth\\"))
pedestrian_groundtruth = generateMatrix(join)

try:
    os.stat('..\\pedestrian_G\\')
except:
    os.mkdir('..\\pedestrian_G')

# Fix ground truth, remove multi-level segmentation 
writeMask("..\\pedestrian_G\\", "pedestrian", pedestrian_groundtruth, 360, 240, 180)

join = map(lambda s: os.path.join("..\\pedestrian_G\\", s), os.listdir("..\\pedestrian_G\\"))
pedestrian_g = generateMatrix(join)

#### Office Video

In [16]:
join = map(lambda s: os.path.join("..\\office_groundtruth\\", s), os.listdir("..\\office_groundtruth\\"))
office_groundtruth = generateMatrix(join)

try:
    os.stat('..\\office_G\\')
except:
    os.mkdir('..\\office_G')

# Fix ground truth, remove multi-level segmentation 
writeMask("..\\office_G\\", "office", office_groundtruth, 360, 240, 180)

join = map(lambda s: os.path.join("..\\office_G\\", s), os.listdir("..\\office_G\\"))
office_g = generateMatrix(join)

## PCP-RPCA with fixed penalty parameter

#### Highway Video

In [17]:
L, S = rpca_pcp(np.transpose(highway), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="highway_pcp")

In [18]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\highway_PCP_L\\', '..\\highway_PCP_S\\', '..\\highway_PCP_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\highway_PCP_L\\", "highway", L.transpose(), 320, 240)
writeImages("..\\highway_PCP_S\\", "highway", S.transpose(), 320, 240)
writeMask("..\\highway_PCP_S_mask\\", "highway", S.transpose(), 320, 240, 10)

join = map(lambda s: os.path.join("..\\highway_PCP_S_mask\\", s), os.listdir("..\\highway_PCP_S_mask\\"))
highway_pcp = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [19]:
### Compare masks 
highway_recall_pcp = recall_score(highway_g, highway_pcp, average='samples')
print(highway_recall_pcp)

highway_precision_pcp = precision_score(highway_g, highway_pcp, average='samples')
print(highway_precision_pcp)

highway_fscore_pcp = f1_score(highway_g, highway_pcp, average='samples')
print(highway_fscore_pcp)

highway_cm_pcp = confusion_matrix(highway_g.ravel(), highway_pcp.ravel())
highway_specificity_pcp = highway_cm_pcp[1,1]/(highway_cm_pcp[1,0] + highway_cm_pcp[1,1])
print(highway_specificity_pcp)

0.2561755803805477
0.3359580194452619
0.2875275712935128
0.23535352855228694


#### Pedestrian Video

In [20]:
L, S = rpca_pcp(np.transpose(pedestrian), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="pedestrian_pcp")

In [21]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\pedestrian_PCP_L\\', '..\\pedestrian_PCP_S\\', '..\\pedestrian_PCP_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\pedestrian_PCP_L\\", "pedestrian", L.transpose(), 360, 240)
writeImages("..\\pedestrian_PCP_S\\", "pedestrian", S.transpose(), 360, 240)
writeMask("..\\pedestrian_PCP_S_mask\\", "pedestrian", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\pedestrian_PCP_S_mask\\", s), os.listdir("..\\pedestrian_PCP_S_mask\\"))
pedestrian_pcp = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [22]:
### Compare masks 
pedestrian_recall_pcp = recall_score(pedestrian_g, pedestrian_pcp, average='samples')
print(pedestrian_recall_pcp)

pedestrian_precision_pcp = precision_score(pedestrian_g, pedestrian_pcp, average='samples')
print(pedestrian_precision_pcp)

pedestrian_fscore_pcp = f1_score(pedestrian_g, pedestrian_pcp, average='samples')
print(pedestrian_fscore_pcp)

pedestrian_cm_pcp = confusion_matrix(pedestrian_g.ravel(), pedestrian_pcp.ravel())
pedestrian_specificity_pcp = pedestrian_cm_pcp[1,1]/(pedestrian_cm_pcp[1,0] + pedestrian_cm_pcp[1,1])
print(pedestrian_specificity_pcp)

0.01843289414464411
0.06231186132499719
0.028215133592309302
0.018940769480705645


#### Office Video

In [23]:
L, S = rpca_pcp(np.transpose(office), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="office_pcp")

In [24]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\office_PCP_L\\', '..\\office_PCP_S\\', '..\\office_PCP_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\office_PCP_L\\", "office", L.transpose(), 360, 240)
writeImages("..\\office_PCP_S\\", "office", S.transpose(), 360, 240)
writeMask("..\\office_PCP_S_mask\\", "office", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\office_PCP_S_mask\\", s), os.listdir("..\\office_PCP_S_mask\\"))
office_pcp = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [25]:
### Compare masks 
office_recall_pcp = recall_score(office_g, office_pcp, average='samples')
print(office_recall_pcp)

office_precision_pcp = precision_score(office_g, office_pcp, average='samples')
print(office_precision_pcp)

office_fscore_pcp = f1_score(office_g, office_pcp, average='samples')
print(office_fscore_pcp)

office_cm_pcp = confusion_matrix(office_g.ravel(), office_pcp.ravel())
office_specificity_pcp = office_cm_pcp[1,1]/(office_cm_pcp[1,0] + office_cm_pcp[1,1])
print(office_specificity_pcp)

0.12989686723829738
0.2572727076836333
0.16873578779927886
0.12887607585528998


## SPCP-RPCA with fixed penalty parameter

#### Highway Video

In [26]:
L, S = rpca_spcp(np.transpose(highway), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="highway_spcp")

In [27]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\highway_SPCP_L\\', '..\\highway_SPCP_S\\', '..\\highway_SPCP_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\highway_SPCP_L\\", "highway", L.transpose(), 320, 240)
writeImages("..\\highway_SPCP_S\\", "highway", S.transpose(), 320, 240)
writeMask("..\\highway_SPCP_S_mask\\", "highway", S.transpose(), 320, 240, 10)

join = map(lambda s: os.path.join("..\\highway_SPCP_S_mask\\", s), os.listdir("..\\highway_SPCP_S_mask\\"))
highway_spcp = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [28]:
### Compare masks 
highway_recall_spcp = recall_score(highway_g, highway_spcp, average='samples')
print(highway_recall_spcp)

highway_precision_spcp = precision_score(highway_g, highway_spcp, average='samples')
print(highway_precision_spcp)

highway_fscore_spcp = f1_score(highway_g, highway_spcp, average='samples')
print(highway_fscore_spcp)

highway_cm_spcp = confusion_matrix(highway_g.ravel(), highway_spcp.ravel())
highway_specificity_spcp = highway_cm_spcp[1,1]/(highway_cm_spcp[1,0]+highway_cm_spcp[1,1])
print(highway_specificity_spcp)

0.24619663378440432
0.41876638861474524
0.30160151984029476
0.22174229212817387


#### Pedestrian Video

In [29]:
L, S = rpca_spcp(np.transpose(pedestrian), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="pedestrian_spcp")

In [30]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\pedestrian_SPCP_L\\', '..\\pedestrian_SPCP_S\\', '..\\pedestrian_SPCP_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\pedestrian_SPCP_L\\", "pedestrian", L.transpose(), 360, 240)
writeImages("..\\pedestrian_SPCP_S\\", "pedestrian", S.transpose(), 360, 240)
writeMask("..\\pedestrian_SPCP_S_mask\\", "pedestrian", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\pedestrian_SPCP_S_mask\\", s), os.listdir("..\\pedestrian_SPCP_S_mask\\"))
pedestrian_spcp = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [31]:
### Compare masks 
pedestrian_recall_spcp = recall_score(pedestrian_g, pedestrian_spcp, average='samples')
print(pedestrian_recall_spcp)

pedestrian_precision_spcp = precision_score(pedestrian_g, pedestrian_spcp, average='samples')
print(pedestrian_precision_spcp)

pedestrian_fscore_spcp = f1_score(pedestrian_g, pedestrian_spcp, average='samples')
print(pedestrian_fscore_spcp)

pedestrian_cm_spcp = confusion_matrix(pedestrian_g.ravel(), pedestrian_spcp.ravel())
pedestrian_specificity_spcp = pedestrian_cm_spcp[1,1]/(pedestrian_cm_spcp[1,0] + pedestrian_cm_spcp[1,1])
print(pedestrian_specificity_spcp)

0.017545992586858062
0.08119195840263553
0.02860198030045817
0.0180329122149806


#### Office Video

In [32]:
L, S = rpca_spcp(np.transpose(office), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="office_spcp")

In [33]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\office_SPCP_L\\', '..\\office_SPCP_S\\', '..\\office_SPCP_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\office_SPCP_L\\", "office", L.transpose(), 360, 240)
writeImages("..\\office_SPCP_S\\", "office", S.transpose(), 360, 240)
writeMask("..\\office_SPCP_S_mask\\", "office", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\office_SPCP_S_mask\\", s), os.listdir("..\\office_SPCP_S_mask\\"))
office_spcp = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [34]:
### Compare masks 
office_recall_spcp = recall_score(office_g, office_spcp, average='samples')
print(office_recall_spcp)

office_precision_spcp = precision_score(office_g, office_spcp, average='samples')
print(office_precision_spcp)

office_fscore_spcp = f1_score(office_g, office_spcp, average='samples')
print(office_fscore_spcp)

office_cm_spcp = confusion_matrix(office_g.ravel(), office_spcp.ravel())
office_specificity_spcp = office_cm_spcp[1,1]/(office_cm_spcp[1,0] + office_cm_spcp[1,1])
print(office_specificity_spcp)

0.13617121852615657
0.2742447810555182
0.17691773175059056
0.135385150200187


## PCP-RPCA with decreasing penalty parameter

#### Highway Video

In [35]:
L, S = rpca_pcp_vary(np.transpose(highway), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="highway_pcp_vary")

In [36]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\highway_PCP_VARY_L\\', '..\\highway_PCP_VARY_S\\', '..\\highway_PCP_VARY_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\highway_PCP_VARY_L\\", "highway", L.transpose(), 320, 240)
writeImages("..\\highway_PCP_VARY_S\\", "highway", S.transpose(), 320, 240)
writeMask("..\\highway_PCP_VARY_S_mask\\", "highway", S.transpose(), 320, 240, 10)

join = map(lambda s: os.path.join("..\\highway_PCP_VARY_S_mask\\", s), os.listdir("..\\highway_PCP_VARY_S_mask\\"))
highway_pcp_vary = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [37]:
### Compare masks 
highway_recall_pcp_vary = recall_score(highway_g, highway_pcp_vary, average='samples')
print(highway_recall_pcp_vary)

highway_precision_pcp_vary = precision_score(highway_g, highway_pcp_vary, average='samples')
print(highway_precision_pcp_vary)

highway_fscore_pcp_vary = f1_score(highway_g, highway_pcp_vary, average='samples')
print(highway_fscore_pcp_vary)

highway_cm_pcp_vary = confusion_matrix(highway_g.ravel(), highway_pcp_vary.ravel())
highway_specificity_pcp_vary = highway_cm_pcp_vary[1,1]/(highway_cm_pcp_vary[1,0] + highway_cm_pcp_vary[1,1])
print(highway_specificity_pcp_vary)

0.2560679795699915
0.3360193914342377
0.2874681844800135
0.23523233030562157


#### Pedestrian Video

In [38]:
L, S = rpca_pcp_vary(np.transpose(pedestrian), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="pedestrian_pcp_vary")

In [39]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\pedestrian_PCP_VARY_L\\', '..\\pedestrian_PCP_VARY_S\\', '..\\pedestrian_PCP_VARY_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\pedestrian_PCP_VARY_L\\", "pedestrian", L.transpose(), 360, 240)
writeImages("..\\pedestrian_PCP_VARY_S\\", "pedestrian", S.transpose(), 360, 240)
writeMask("..\\pedestrian_PCP_VARY_S_mask\\", "pedestrian", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\pedestrian_PCP_VARY_S_mask\\", s), os.listdir("..\\pedestrian_PCP_VARY_S_mask\\"))
pedestrian_pcp_vary = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [40]:
### Compare masks 
pedestrian_recall_pcp_vary = recall_score(pedestrian_g, pedestrian_pcp_vary, average='samples')
print(pedestrian_recall_pcp_vary)

pedestrian_precision_pcp_vary = precision_score(pedestrian_g, pedestrian_pcp_vary, average='samples')
print(pedestrian_precision_pcp_vary)

pedestrian_fscore_pcp_vary = f1_score(pedestrian_g, pedestrian_pcp_vary, average='samples')
print(pedestrian_fscore_pcp_vary)

pedestrian_cm_pcp_vary = confusion_matrix(pedestrian_g.ravel(), pedestrian_pcp_vary.ravel())
pedestrian_specificity_pcp_vary = pedestrian_cm_pcp_vary[1,1]/(pedestrian_cm_pcp_vary[1,0] + pedestrian_cm_pcp_vary[1,1])
print(pedestrian_specificity_pcp_vary)

0.01843289414464411
0.062414346376454335
0.028225553591356038
0.018940769480705645


#### Office Video

In [41]:
L, S = rpca_pcp_vary(np.transpose(office), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="office_pcp_vary")

In [42]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\office_PCP_VARY_L\\', '..\\office_PCP_VARY_S\\', '..\\office_PCP_VARY_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)

writeImages("..\\office_PCP_VARY_L\\", "office", L.transpose(), 360, 240)
writeImages("..\\office_PCP_VARY_S\\", "office", S.transpose(), 360, 240)
writeMask("..\\office_PCP_VARY_S_mask\\", "office", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\office_PCP_VARY_S_mask\\", s), os.listdir("..\\office_PCP_VARY_S_mask\\"))
office_pcp_vary = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [43]:
### Compare masks 
office_recall_pcp_vary = recall_score(office_g, office_pcp_vary, average='samples')
print(office_recall_pcp_vary)

office_precision_pcp_vary = precision_score(office_g, office_pcp_vary, average='samples')
print(office_precision_pcp_vary)

office_fscore_pcp_vary = f1_score(office_g, office_pcp_vary, average='samples')
print(office_fscore_pcp_vary)

office_cm_pcp_vary = confusion_matrix(office_g.ravel(), office_pcp_vary.ravel())
office_specificity_pcp_vary = office_cm_pcp_vary[1,1]/(office_cm_pcp_vary[1,0] + office_cm_pcp_vary[1,1])
print(office_specificity_pcp_vary)

0.1296793615630713
0.25729179290528553
0.16856500762987742
0.12866510033324544


## SPCP-RPCA with decreasing penalty parameter

#### Highway Video

In [46]:
L, S = rpca_spcp_vary(np.transpose(highway), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="highway_spcp_vary")

In [47]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\highway_SPCP_VARY_L\\', '..\\highway_SPCP_VARY_S\\', '..\\highway_SPCP_VARY_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)
        
writeImages("..\\highway_SPCP_VARY_L\\", "highway", L.transpose(), 320, 240)
writeImages("..\\highway_SPCP_VARY_S\\", "highway", S.transpose(), 320, 240)
writeMask("..\\highway_SPCP_VARY_S_mask\\", "highway", S.transpose(), 320, 240, 10)

join = map(lambda s: os.path.join("..\\highway_SPCP_VARY_S_mask\\", s), os.listdir("..\\highway_SPCP_VARY_S_mask\\"))
highway_spcp_vary = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [48]:
### Compare masks 
highway_recall_spcp_vary = recall_score(highway_g, highway_spcp_vary, average='samples')
print(highway_recall_spcp_vary)

highway_precision_spcp_vary = precision_score(highway_g, highway_spcp_vary, average='samples')
print(highway_precision_spcp_vary)

highway_fscore_spcp_vary = f1_score(highway_g, highway_spcp_vary, average='samples')
print(highway_fscore_spcp_vary)

highway_cm_spcp_vary = confusion_matrix(highway_g.ravel(), highway_spcp_vary.ravel())
highway_specificity_spcp_vary = highway_cm_spcp_vary[1,1]/(highway_cm_spcp_vary[1,0] + highway_cm_spcp_vary[1,1])
print(highway_specificity_spcp_vary)

0.2460251341374694
0.41960111416373613
0.30164239244177776
0.2215941609378051


#### Pedestrian Video

In [49]:
L, S = rpca_spcp_vary(np.transpose(pedestrian), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="pedestrian_spcp_vary")

In [50]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\pedestrian_SPCP_VARY_L\\', '..\\pedestrian_SPCP_VARY_S\\', '..\\pedestrian_SPCP_VARY_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)
        
writeImages("..\\pedestrian_SPCP_VARY_L\\", "pedestrian", L.transpose(), 360, 240)
writeImages("..\\pedestrian_SPCP_VARY_S\\", "pedestrian", S.transpose(), 360, 240)
writeMask("..\\pedestrian_SPCP_VARY_S_mask\\", "pedestrian", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\pedestrian_SPCP_VARY_S_mask\\", s), os.listdir("..\\pedestrian_SPCP_VARY_S_mask\\"))
pedestrian_spcp_vary = generateMatrix(join)

#### Calculate F-score, precision, recall, specificity

In [51]:
### Compare masks 
pedestrian_recall_spcp_vary = recall_score(pedestrian_g, pedestrian_spcp_vary, average='samples')
print(pedestrian_recall_spcp_vary)

pedestrian_precision_spcp_vary = precision_score(pedestrian_g, pedestrian_spcp_vary, average='samples')
print(pedestrian_precision_spcp_vary)

pedestrian_fscore_spcp_vary = f1_score(pedestrian_g, pedestrian_spcp_vary, average='samples')
print(pedestrian_fscore_spcp_vary)

pedestrian_cm_spcp_vary = confusion_matrix(pedestrian_g.ravel(), pedestrian_spcp_vary.ravel())
pedestrian_specificity_spcp_vary = pedestrian_cm_spcp_vary[1,1]/(pedestrian_cm_spcp_vary[1,0] + pedestrian_cm_spcp_vary[1,1])
print(pedestrian_specificity_spcp_vary)

0.017545992586858062
0.08119195840263553
0.02860198030045817
0.0180329122149806


#### Office Video

In [52]:
L, S = rpca_spcp_vary(np.transpose(office), lambda0="default", convergenceTolerance=10**(-3), convergenceLog="office_spcp_vary")

In [53]:
# Create folders for low-rank frames, sparse frames and binary mask frames

for folder in ['..\\office_SPCP_VARY_L\\', '..\\office_SPCP_VARY_S\\', '..\\office_SPCP_VARY_S_mask\\']:
    try:
        os.stat(folder)
    except:
        os.mkdir(folder)
    filelist = [ f for f in os.listdir(folder)]
    for f in filelist:
        os.remove(folder + f)
        
writeImages("..\\office_SPCP_VARY_L\\", "office", L.transpose(), 360, 240)
writeImages("..\\office_SPCP_VARY_S\\", "office", S.transpose(), 360, 240)
writeMask("..\\office_SPCP_VARY_S_mask\\", "office", S.transpose(), 360, 240, 10)

join = map(lambda s: os.path.join("..\\office_SPCP_VARY_S_mask\\", s), os.listdir("..\\office_SPCP_VARY_S_mask\\"))
office_spcp_vary = generateMatrix(join)

#### Calculate F-measure, precision, recall, specificity

In [54]:
### Compare masks 
office_recall_spcp_vary = recall_score(office_g, office_spcp_vary, average='samples')
print(office_recall_spcp_vary)

office_precision_spcp_vary = precision_score(office_g, office_spcp_vary, average='samples')
print(office_precision_spcp_vary)

office_fscore_spcp_vary = f1_score(office_g, office_spcp_vary, average='samples')
print(office_fscore_spcp_vary)

office_cm_spcp_vary = confusion_matrix(office_g.ravel(), office_spcp_vary.ravel())
office_specificity_spcp_vary = office_cm_spcp_vary[1,1]/(office_cm_spcp_vary[1,0] + office_cm_spcp_vary[1,1])
print(office_specificity_spcp_vary)

0.13607314439542242
0.2749656659033363
0.17699813834462955
0.1352856560619501
